Let's start with re. making the grocery list. If we'd make this into an app of some kind then eh, but for now we'll do it pure python...

In [1]:
import pandas as pd

In [2]:
# df = pd.DataFrame(data={
#     'name':[]
#     ,'quantity':[]
#     ,'cold':[]
#     ,'location':[]
#     ,'comment':[]
# })

# df

In [3]:
# df.to_csv('data/groc_list.csv', index=False)

Now that we have the initial structure set up, let's get going.

In [4]:
# df = pd.read_csv('data/groc_list.csv', escapechar='\t', delimiter=';')
df = pd.read_csv('data/groc_list.csv')
print(df.shape)
df.head()

(37, 5)


,name,quantity,cold,location,comment
0,eggs,1,yes,10,NaN
1,milk,1,yes,10,NaN
2,beer,2,?,?,NaN
3,cheese,2,yes,7,NaN
4,mixed nuts,1,no,9,NaN


In [5]:
#Oh wait, w do have some datacleaning with ?... cold I can work around as we're essentially using that for boolean,
#but lcoation... we'll have to change.. 

#At first we'll try replacing it with an arbitarily big number.If that doesn't work per mapping purposes later on theen
#we can just replace it 

In [6]:
df['location'] = df['location'].str.replace('?', '9001')
df['location'] = df['location'].astype(int)

In [7]:
# df.to_csv('data/groc_list.csv')

We had 2 features in VBA:

The first was pragmatically adding something to the list, and in all honesty the interface in VBA was quite nice and that aspect is kinda unneeded, base motivation wise to get updated, however we will anyways.

Note that I honestly think SQL would be a great use for this, making a query checking if something is LIKE the others. However, this is good enough as is.

The second, and something I think Python will be better at, will be organizing the list. And, this is what I'm most looking forward to so I'll start with it.

Albeit we're assuming a base assumption of the quadrants,a nd arguably we know that from the start.

Ie let's reiterate the following (which I think I did a different assumption last time):

Note that a quadrant is roughly equivelant to 1-4 isles parralel. In the main grocery store that I go to the back section is big enough that I give that area its own zone; the random displays before the cashiers however...

M1 is to view the base quadrants starting from the back left, going from one end of the row to its end, before starting again on the second row. Ie how a regular matrix works.<br>This is how I originally grouped them.

M2 is to start from the front left, like starting from A1 with the white queen's rook in a gameof chess, going from left to right... before starting the next row from A2, that rook's pawn.

I think M2 is more intuitive and will start like that.

## Bear in mind
That I'll be using my grocery store in mind, which is a 3x4, ultimately deciding to do M2. Hence, the base locations:<br>
9,10,11,12<br>
5,6,7,8<br>
1,2,3,4<br>

In [8]:
#The assumption is to walk down isles. Working with that, the working assumption is that one enters the store and in the
#direction of the entrance there are already isles found; as opposed to having to make some perpendicular turn.

# def orderplanner_base(rows, columns):
#     basecolumn_names = []
#     for i in range(columns):
#         basecolumn_names.append(f"{i+1}_column")
#     for c in range(len(basecolumn_names)):
#         basecolumn_names[c] = [c + 1+ columns * i for i in range(rows)]
#         print(f"basecolumn_names[{c}]")
#         print(basecolumn_names[c])
#         print("~"*45)
    
#     return 

In [9]:
# orderplanner_base(3, 4)

1,5,9,,2610,3711,4,8,12

In [10]:
#Good. Now let's rearrange this. We'll keep thigs simple and assume you enter through either on the "front" left or right
#of the store; no noob, don't be lazy and make it any... We'll at least simplify it to one of the corners; nah, let's say
#it's a giant store...; gah, we'll keep it simple for now and just do one of the two... Plus, even if it is giant, realistically
#all of hte cash registers arel ikely in one area...Hmm, that could also be a thing to customize later - variable cash
#register display.



In [11]:
#We'll code for left or right, ie 1 or 4/w/e the bigger number is.

def orderplanner_purchase(rows, columns, entrance, cold=True):
    basecolumn_names = []
    base_nos = []
    for i in range(columns):
        basecolumn_names.append(f"{i+1}_column")
    for c in range(len(basecolumn_names)):
        basecolumn_names[c] = [c + 1 + columns * i for i in range(rows)]
    for i in basecolumn_names:
        for j in i:
            base_nos.append(j)
    #[1, 5, 9, 2, 6, 10, 3, 7, 11, 4, 8, 12]
    
    #Used to hav something much more complciated, but realized could simply do this, as we'll be needed
    #these guys still grouped to make it easier
    
    #In retrospect could do something like within z doors of 1... but eh, this is fine for now.
    if entrance != 1:
        basecolumn_names = basecolumn_names[::-1]
     
    zone_order = [c for c in basecolumn_names]
    
    #Hmm, actually it'll be easier to do cold first:
    defaults = range(1, rows*columns + 1) #Relevant later on, but logically better to do it now.
    
    #Recall how we have a ? for certain thigns.... plus pragmatically we should be stingent in cases of health. Henc
    #changing ==True to != False
    if cold != False:
        for c in basecolumn_names[::-1]:
            zone_order.append([bro + rows*columns for bro in c]) #Recall that we need to adjust this for the mapping later,
            #as otherwise we'd just map the second half, as we have repeats of zone order...
        defaults = range(1, 2*rows*columns + 1) #Relevant later on
    
    for i in range(len(zone_order)):
        if i % 2 == 1:
            zone_order[i] = zone_order[i][::-1]
    
    #Now we'll just make them into a list:
    shopping_order = []
    for i in zone_order:
        for j in range(rows):
            shopping_order.append(i[j])
    #So too for zon order, as recall that those are made up of 1,2 * columns of {row} zones:
    
    
    #Now let's do a mapping:
    #Well, pragmaticaly we'll be using this in Pandas, but eh, let's make the dictionary:
    
    #Note that we'd just add + r*c to the base of the cold item.
    
    shopping_dictionary = {shopping_order[i]:defaults[i] for i in range(len(shopping_order))}
    #Hmm, defaults are fine, but re. shopping order...
    
    return shopping_dictionary

In [12]:
current_order = orderplanner_purchase(3, 4, 4, cold=True)

In [13]:
current_order

{4: 1,
 8: 2,
 12: 3,
 11: 4,
 7: 5,
 3: 6,
 2: 7,
 6: 8,
 10: 9,
 9: 10,
 5: 11,
 1: 12,
 13: 13,
 17: 14,
 21: 15,
 22: 16,
 18: 17,
 14: 18,
 15: 19,
 19: 20,
 23: 21,
 24: 22,
 20: 23,
 16: 24}

In [14]:
#Now lt's apply it:
df.head()

,name,quantity,cold,location,comment
0,eggs,1,yes,10,NaN
1,milk,1,yes,10,NaN
2,beer,2,?,9001,NaN
3,cheese,2,yes,7,NaN
4,mixed nuts,1,no,9,NaN


In [15]:
conditions = [
    df['cold'] == 'no'
    ,df['cold'] != 'no'
]
results = [
    df['location']
    ,df['location']+12
]

In [16]:
#Hmm, this might be tricki than I thought

df['order'] = [df['location'][i] if df['cold'][i] != 'no' else df['location'][i] + 12 for i in range(len(df['location']))]

In [17]:
df.head()

,name,quantity,cold,location,comment,order
0,eggs,1,yes,10,NaN,10
1,milk,1,yes,10,NaN,10
2,beer,2,?,9001,NaN,9001
3,cheese,2,yes,7,NaN,7
4,mixed nuts,1,no,9,NaN,21


Now, if we have 1 as an entrance then we'd keep this 'base' order, but since we have 4:

And, note that we'd "want" it in this following:
4812,,3711,,2610,,159

Anyways, we got the mapping done. The idea to be would be to apply it to whatever the final pandas dataframe is, sorting it by that.

Hmm, so what's the next step? Guess now let's work on somehow iterating in Python through each item, asking if we want it.

M1  - Figure that out somehow, ie like what we had in VBA. Ie where we go through each item and ask if we need it...

M2 - Given a list of items, choose just those bros...

At the same time, we'd like to then filter out those guys and redo m1... ie yeah, one thing to pre-gam the list, but it's like - who says you didn't miss out on something lse?!?!!? Plus, another disadvanag is that we'd then have to pre=-write the quantities somehow or still go back though those and update them for the quantities...

At the moment, I'm inclined to say just do M1 and we'll print out the default quantity, not eve bothering to ask for the quantity; no, perhaps have an optional thing for one; regardless, don't focus on quantity but just items, going through ach bro, asking if you want it.

In [18]:
i = -1
current_list = []

In [19]:
i += 1
df['name'][i]

'eggs'

In [20]:
#Fututr streamlit code for variable input to be like if yes or whatever, perhaps if True:

In [21]:
#Then I suppose we'd make a new table.Hmm...how to do that?
#m1 start wih just hte nam, and we'll drag them from the original later
    #Yeah, hat seems like the best. Plus, I honestly would still like more work with cmbining

#m2, there still is a certain simplcity of grabing all that information now.

decision = True
if decision == True:
    current_list.append(df['name'][i])

In [22]:
grocery_list = pd.DataFrame(data={'name':current_list})

In [23]:
grocery_list = grocery_list.join(df.set_index('name'), on=['name'], how='left') #Seems like we always have to set the indexes...
#Albit we don't have to formally do it on both, as hte on paramter does that, but effectively we still are...
grocery_list

,name,quantity,cold,location,comment,order
0,eggs,1,yes,10,NaN,10


In [24]:
#Oh, make sure we actually applied the order... was wondering why this was off - as this s still per quadrans..

In [25]:
grocery_list.sort_values(by=['location']).head(df.shape[0])[['name', 'quantity']]

,name,quantity
0,eggs,1


Now that we finished this, we're onto the next step of generating the list... Looks like we're going back to Streamlit


Actually, let's go back a few steps:

Recall that we have two parts of the problem:

The first is maintaining/updating the list. Included in that is making sure no duplicate information gets put in, store is correct, etc.

The second is pulling from that whatever items currently desired.

@@@@@@@@@@@@@@@@@@@@@@

Honestly, the first part is ideal in Excel... yeah, potentially I could make something better, but it's like why bother...

Re. the second part - yeah,potentally a web-app would be better, ie something boss with streamlit that would let me show the list and manually click whatever I need then refresh, assuming it can even do something like that, but the effort...

Ie I'd like that more than the current 1 at a time in Excel, but it's like it aint broke... plus it forces me to go through each one.... Now, while albeit the current list is only needed for Jewel; oh, never mind - regardless of the format, that column would need to change to accomodate the current main grocery store that I go to...

So, unfortunately this project is being frozen indefinitely (4/3/24)... While I do find merit in completing it, I just have no motiviation to do so when my Excel/VBA version works fine. Plus, I'll still keep the minor victory of that somewhat complex zone-to-order conversion formula.